In [18]:
# %load jsonToDF.py
import pandas as pd
import os

all_files = os.listdir("years")
lst = []
for json_file in all_files:
    df = pd.read_json("years/"+json_file, orient='index')
    lst.append(df)
frame = pd.concat(lst, axis = 1)
#frame.to_csv("all_years.csv",sep='\t',encoding='utf-8')

In [12]:
df = pd.read_json("years/"+all_files[0], orient='index')

In [19]:
frame

,0,1,2,3,4,5,6,7,0,1,...,66,67,68,69,70,71,72,73,74,75
artist,Nat King Cole,Andrews Sisters,Patti Page,Tennessee Ernie Ford,Patti Page,Perry Como,Bing Crosby,Frankie Laine,Nat King Cole,Tony Bennett,...,George Ezra,Sia,Kelly Clarkson,Ed Sheeran,Ella Henderson,Alessia Cara,Mr. Probz,Ne-Yo,One Direction,Calvin Harris
difficult_words,13,40,5,10,9,22,12,15,4,6,...,13,17,16,42,14,42,15,16,31,12
f_k_grade,2.9,3.3,2.9,0.9,6,0.1,2.1,0.1,0.1,2.5,...,0.5,1.7,0.1,1.3,1.7,2.1,1.7,0.1,2.9,-0.3
flesch_index,88.74,82.31,88.74,99.23,69.79,106.67,90.77,106.67,106.67,89.75,...,100.24,97.2,106.67,98.21,97.2,96.18,91.78,106.67,83.32,107.69
fog_index,5.2,4.4,5.2,4.4,6,4.8,4.4,4.8,4.8,4.8,...,4,5.2,4.8,4.8,5.2,5.6,4,4.8,4,4.4
lyrics,"Mona Lisa, Mona Lisa, men have named you\nYou'...",I wanna be Loved\nBy Andrews Sisters\n\nOooo-o...,I was dancing with my darling to the Tennessee...,Each time I hold someone new\nMy arms grow col...,"Unfortunately, we are not licensed to display ...","Hoop dee doo, hoop dee doo, \nI hear a polka ...",i love those dear hearts and gentle people\nwh...,"My heart knows what the wild goose knows,\nI m...",They try to tell us we're too young\nToo young...,Because of you there's a song in my heart\nBec...,...,My house in Budapest\nMy hidden treasure chest...,Party girls don't get hurt\nCan't feel anythin...,This is my heartbeat song and I'm gonna play i...,"I met this girl late last year\nShe said ""Don'...",I keep going to the river to pray\n'Cause I ne...,(I guess right now you've got the last laugh)\...,My face above the water\nMy feet can't touch t...,You know from the moment she turned around\nSh...,Going out tonight\nChanges into something red\...,I want you to breathe me in\nLet me be your ai...
num_dupes,10,32,10,3,0,22,17,18,5,5,...,55,33,38,31,37,13,8,21,34,38
num_lines,17,31,16,18,3,54,39,40,13,13,...,53,46,56,77,53,51,22,26,52,52
num_syllables,189,270.9,174.6,135.9,46.8,423,303.3,350.1,100.8,131.4,...,323.1,453.6,471.6,686.7,501.3,570.6,169.2,217.8,399.6,378
num_words,145,189,138,117,32,382,239,306,92,101,...,272,378,433,594,435,469,126,193,296,340


In [22]:
zz=pd.read_csv('all_years.csv')

In [23]:
zz

,Unnamed: 0,0,1,2,3,4,5,6,7,0.1,...,66.35,67.32,68.32,69.30,70.26,71.23,72.21,73.20,74.20,75.18
0,artist,Nat King Cole,Andrews Sisters,Patti Page,Tennessee Ernie Ford,Patti Page,Perry Como,Bing Crosby,Frankie Laine,Nat King Cole,...,George Ezra,Sia,Kelly Clarkson,Ed Sheeran,Ella Henderson,Alessia Cara,Mr. Probz,Ne-Yo,One Direction,Calvin Harris
1,difficult_words,13,40,5,10,9,22,12,15,4,...,13,17,16,42,14,42,15,16,31,12
2,f_k_grade,2.9,3.3,2.9,0.9,6.0,0.1,2.1,0.1,0.1,...,0.5,1.7000000000000002,0.1,1.3,1.7000000000000002,2.1,1.7000000000000002,0.1,2.9,-0.30000000000000004
3,flesch_index,88.74,82.31,88.74,99.23,69.79,106.67,90.77,106.67,106.67,...,100.24,97.2,106.67,98.21,97.2,96.18,91.78,106.67,83.32,107.69
4,fog_index,5.2,4.4,5.2,4.4,6.0,4.800000000000001,4.4,4.800000000000001,4.800000000000001,...,4.0,5.2,4.800000000000001,4.800000000000001,5.2,5.6,4.0,4.800000000000001,4.0,4.4
5,lyrics,"Mona Lisa, Mona Lisa, men have named you\r\nYo...",I wanna be Loved\r\nBy Andrews Sisters\r\n\r\n...,I was dancing with my darling to the Tennessee...,Each time I hold someone new\r\nMy arms grow c...,"Unfortunately, we are not licensed to display ...","Hoop dee doo, hoop dee doo, \r\nI hear a polk...",i love those dear hearts and gentle people\r\n...,"My heart knows what the wild goose knows,\r\nI...",They try to tell us we're too young\r\nToo you...,...,My house in Budapest\r\nMy hidden treasure che...,Party girls don't get hurt\r\nCan't feel anyth...,This is my heartbeat song and I'm gonna play i...,"I met this girl late last year\r\nShe said ""Do...",I keep going to the river to pray\r\n'Cause I ...,(I guess right now you've got the last laugh)\...,My face above the water\r\nMy feet can't touch...,You know from the moment she turned around\r\n...,Going out tonight\r\nChanges into something re...,I want you to breathe me in\r\nLet me be your ...
6,num_dupes,10,32,10,3,0,22,17,18,5,...,55,33,38,31,37,13,8,21,34,38
7,num_lines,17,31,16,18,3,54,39,40,13,...,53,46,56,77,53,51,22,26,52,52
8,num_syllables,189.0,270.9,174.6,135.9,46.8,423.0,303.3,350.1,100.8,...,323.1,453.6,471.6,686.7,501.3,570.6,169.2,217.8,399.6,378.0
9,num_words,145,189,138,117,32,382,239,306,92,...,272,378,433,594,435,469,126,193,296,340
